In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd

print('Finish importing package')

driver = webdriver.Chrome()

Finish importing package


- Các hàm dùng để lấy các dữ liệu cần thiết.

In [6]:
def getName(hotel):
  HOTEL_NAME = hotel.find('div', class_='fcab3ed991 a23c043802')
  if(HOTEL_NAME == None):
    return '-'
  else:
    return HOTEL_NAME.text

def getRating(hotel):
  HOTEL_RATING = hotel.find('div', class_='b5cd09854e d10a6220b4')
  if(HOTEL_RATING == None):
    return '-'
  else:
    return HOTEL_RATING.text

def getReviews(hotel):
  HOTEL_REVIEWS = hotel.find('div', class_='d8eab2cf7f c90c0a70d3 db63693c62')
  if(HOTEL_REVIEWS == None):
    return '-'
  else:
    # 99 reviews, 1 review, 34 external
    return HOTEL_REVIEWS.text.replace(' reviews', '').replace(' review', '').replace(' external', '')

def getLocation(hotel):
  HOTEL_LOCATION = hotel.find('span', class_='f4bd0794db b4273d69aa')
  if(HOTEL_LOCATION == None):
    return '-'
  else:
    return HOTEL_LOCATION.text

def getPrice(hotel):
  HOTEL_PRICE = hotel.find('span', class_='fcab3ed991 bd73d13072')
  if(HOTEL_PRICE == None):
    return '-'
  else:
    # VND\xa0500000
    return HOTEL_PRICE.text.replace('VND\xa0', '')

- Crawl data từ Booking.com

In [7]:
# data url
URL_HCM = 'https://www.booking.com/searchresults.en-gb.html?ss=Ho+Chi+Minh+City&ssne=Ho+Chi+Minh+City&ssne_untouched=Ho+Chi+Minh+City&label=gen173nr-1FCAEoggI46AdIM1gEaPQBiAEBmAEJuAEZyAEM2AEB6AEB-AELiAIBqAIDuALWktKUBsACAdICJGFhZTc0NmNjLTk2ZGUtNDZkNi1iNTJhLWE3N2UzYmRlYTNjONgCBuACAQ&sid=4014c954f7c62c8adf4f7ab4f5a3445d&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=-3730078&dest_type=city&checkin=2022-06-20&checkout=2022-06-21&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure'

# 1 page has 25 hotel
NUMBER_OF_PAGES = 40

def crawlData():
  res = []

  for i in range(NUMBER_OF_PAGES):
    offset = i*25
    url = URL_HCM + '&offset=' + str(offset)
    driver.get(url)
    page_source = BeautifulSoup(driver.page_source)

    hotels = page_source.find_all('div', class_='a826ba81c4 fe821aea6c fa2f36ad22 afd256fc79 d08f526e0d ed11e24d01 ef9845d4b3 da89aeb942')

    # In dữ liệu ra console và đưa dữ liệu vào các mảng
    for hotel in hotels:
      hotel_name = getName(hotel)
      hotel_price = getPrice(hotel)
      hotel_rating = getRating(hotel)
      hotel_review = getReviews(hotel)
      hotel_locate = getLocation(hotel)
      
      res.append([hotel_name, hotel_price, hotel_rating, hotel_review, hotel_locate])
  return res
hotels_data = crawlData()
print(len(hotels_data))

1000


- Export to csv file

In [8]:
pd.DataFrame(hotels_data).to_csv('raw data.csv', header = ['Name', 'Price', 'Rating', 'Reviews', 'Location'])